# Project 2

## Exploring the Relationship Between NYC Subway Ridership and COVID-19 Cases

In this project, I explore how the COVID-19 pandemic affected public mobility in New York City by comparing two real-world datasets from NYC Open Data:

COVID-19 Daily Case Counts
Source: https://data.cityofnewyork.us/Health/COVID-19-Daily-Counts/rc75-m7u3

MTA Daily Subway Ridership (2020–2025)
Source: https://data.ny.gov/Transportation/MTA-Daily-Ridership-Data-2020-2025/vxuj-8kew

The goal of this analysis is to understand whether COVID infection trends correlate with subway usage, and how this relationship changes depending on the time scale:
daily, weekly, and monthly.

I will first clean and load the datasets, then build several visualizations to compare trends. Along the way, I explain each step, the reasoning behind it, and what the graphs reveal.

In [57]:
# ensure the visualizations render properly across VSCode, Jupyter Book, etc.
# https://plotly.com/python/renderers/

import plotly.io as pio

pio.renderers.default = "notebook_connected+plotly_mimetype"

In [26]:
import pandas as pd
import matplotlib.pyplot as plt


## Loading and Preparing the MTA Subway Ridership Data

Before comparing ridership with COVID cases, I first load the MTA daily ridership dataset.
This section cleans the date column and converts the ridership numbers (which include commas) into proper numeric values.

In [34]:
import pandas as pd

# Load MTA ridership data
mta = pd.read_csv("MTA_Daily_Ridership_Data__2020_-_2025_20251121 (1).csv")

# Convert Date to datetime
mta["DATE"] = pd.to_datetime(mta["Date"], format="%m/%d/%Y")

# Convert subway ridership to numeric
mta["subways_total"] = (
    mta["Subways: Total Estimated Ridership"]
    .astype(str)
    .str.replace(",", "")
    .astype(float)
)

# Keep only DATE + subway ridership
mta_clean = mta[["DATE", "subways_total"]]

mta_clean.head()


,DATE,subways_total
0,2020-03-01,2212965.0
1,2020-03-02,5329915.0
2,2020-03-03,5481103.0
3,2020-03-04,5498809.0
4,2020-03-05,5496453.0


## Loading the COVID-19 Daily Case Dataset

Now I load and prepare the COVID case data.
The dataset includes daily case counts for NYC starting in early 2020.
Here I convert the date column into a datetime format and keep only the fields needed for analysis.

In [56]:
import pandas as pd
import matplotlib.pyplot as plt
covid = pd.read_csv("COVID-19_Daily_Counts_of_Cases,_Hospitalizations,_and_Deaths_20251121.csv")
covid.head()

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,...,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,INCOMPLETE
0,02/29/2020,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,03/01/2020,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,03/02/2020,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,03/03/2020,1,0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,03/04/2020,5,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Aligning the Date Ranges

The two datasets begin and end on slightly different dates.
Before merging them, I restrict the COVID data to the date range covered by the MTA ridership dataset, so that the merged dataset only contains dates present in both sources.

In [55]:
# Align both datasets to the same DATE range
start_date = mta_clean["DATE"].min()
end_date = mta_clean["DATE"].max()

covid_clean = covid_clean[(covid_clean["DATE"] >= start_date) &
                          (covid_clean["DATE"] <= end_date)]



## Merging the Two Datasets

Now that both datasets are cleaned and aligned by date, I merge them into a single dataframe.
This allows me to analyze subway ridership and COVID cases side by side for every date from 2020 to 2025.

In [36]:
# Merge on DATE
merged = pd.merge(mta_clean, covid_clean, on="DATE", how="inner")

merged.head()


,DATE,subways_total,CASE_COUNT
0,2020-03-01,2212965.0,0
1,2020-03-02,5329915.0,0
2,2020-03-03,5481103.0,1
3,2020-03-04,5498809.0,5
4,2020-03-05,5496453.0,3


## Making Sure the Data Types Are Numeric

Before plotting or computing statistics, I want to make sure both key columns are stored as numeric types.
Here I explicitly convert `subways_total` and `CASE_COUNT` to numeric and drop any rows where either one is missing.
This prevents subtle bugs and ensures the plots and correlation calculations behave as expected.


In [38]:
# Ensure both columns are numeric
merged["subways_total"] = pd.to_numeric(merged["subways_total"], errors="coerce")
merged["CASE_COUNT"] = pd.to_numeric(merged["CASE_COUNT"], errors="coerce")

# Optionally drop rows where both are missing
merged = merged.dropna(subset=["subways_total", "CASE_COUNT"])


## Visualizing Correlation Between Ridership and COVID Cases

First, plotting COVID cases and subway ridership on the same Y-axis.
The intention is to see whether both lines move together or in opposite directions.

But the visualization fails to show anything useful.
Why?

Because the scale of subway ridership (hundreds of thousands to millions) is far larger than the scale of COVID cases (hundreds to thousands).
As a result, the COVID line is nearly invisible.

This is an important realization: raw counts cannot be meaningfully compared on the same axis when their magnitudes differ dramatically.

So in the next step, I adjust the approach.

In [39]:
import plotly.express as px

fig = px.line(
    merged,
    x="DATE",
    y=["subways_total", "CASE_COUNT"],
    title="NYC Subway Ridership vs COVID-19 Daily Case Counts (2020–2025)",
    labels={"value": "Count", "variable": "Data Series"},
)

fig.show()


## A Better Daily Visualization Using Two Y-Axes

To fix the scale issue, I plot the two series on separate Y-axes while sharing the same X-axis (date).
This allows each series to keep its own natural unit scale while still letting us see how they move together over time.

Takeaway #1 (Daily Pattern):

There is a clear inverse relationship.

When COVID cases spike, subway ridership drops sharply.

When COVID cases fall, ridership rises again.

This reflects immediate behavioral responses:
New Yorkers avoided the subway during major waves and returned as perceived risk decreased.

In [40]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=merged["DATE"],
        y=merged["subways_total"],
        name="Subway Ridership"
    ),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=merged["DATE"],
        y=merged["CASE_COUNT"],
        name="COVID-19 Cases"
    ),
    secondary_y=False,
)

fig.update_layout(
    title_text="NYC Subway Ridership vs COVID-19 Daily Cases (2020–2025)"
)

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="COVID-19 Daily Cases", secondary_y=False)
fig.update_yaxes(title_text="Subway Ridership", secondary_y=True)

fig.show()


## Quantifying the Daily Relationship with Correlation

The plots already suggest an inverse relationship between COVID cases and ridership at the daily level.
To back this up mathematically, I compute the Pearson correlation coefficient between `CASE_COUNT` and `subways_total`.

A negative value here confirms that, on days when COVID cases are higher, subway ridership tends to be lower.


In [51]:
corr_daily = merged[["subways_total", "CASE_COUNT"]].corr().iloc[0, 1]
corr_daily


np.float64(-0.0740774960154273)

## Resampling the Data: Broader Trends

Daily data is helpful for observing immediate reactions, but it can be noisy.  
In this section, we resample the merged dataset by **week**, computing the sum for each week.

Weekly aggregation smooths out day-to-day spikes and lets us observe medium-term trends in mobility and infections.


In [41]:
merged_ts = merged.sort_values("DATE").set_index("DATE")


In [47]:
# Weekly totals (W = weekly, end of week, default Sunday)
weekly = merged_ts.resample("W").sum().reset_index()

# Monthly totals
monthly = merged_ts.resample("M").sum().reset_index()


/var/folders/4r/82c5nm0x3gqd56z9s4z1125m0000gn/T/ipykernel_61958/1911899917.py:5: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



## A Helper Function for Weekly and Monthly Plots

To avoid repeating the same plotting code for weekly and monthly data,
I define a small helper function `plot_dual`. It takes a dataframe and a title, and produces a dual-axis line chart with:

- COVID-19 cases on the left Y-axis
- Subway ridership on the right Y-axis

I’ll reuse this function to visualize weekly and monthly trends side by side.


In [48]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_dual(df, title):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Scatter(
            x=df["DATE"],
            y=df["CASE_COUNT"],
            name="COVID-19 Cases",
            line=dict(color="red")
        ),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(
            x=df["DATE"],
            y=df["subways_total"],
            name="Subway Ridership",
            line=dict(color="blue")
        ),
        secondary_y=True,
    )

    fig.update_layout(
        title_text=title,
        legend=dict(x=0.01, y=0.99),
    )

    fig.update_xaxes(title_text="Date")
    fig.update_yaxes(title_text="COVID-19 Cases", secondary_y=False)
    fig.update_yaxes(title_text="Subway Ridership", secondary_y=True)

    fig.show()


## Weekly Comparison of Subway Ridership and COVID Cases

This visualization shows the weekly totals of ridership and COVID cases on dual axes.  
Weekly data smooths out many of the sharp fluctuations seen in the daily chart.

Interestingly, the weekly trend reveals periods where both ridership and COVID cases appear to rise together, especially in 2021 and early 2022.

We discuss later why this happens and what it means.


In [49]:
plot_dual(weekly, "Weekly NYC Subway Ridership vs COVID-19 Cases (Totals)")


## Monthly Comparison of Ridership and COVID Cases

The monthly chart shows broad, long-term trends.  
Here we can see NYC’s gradual reopening process from 2020 to 2023.

Because monthly aggregation blends pre-lockdown and post-lockdown periods within the same month,  
the apparent correlation shifts — sometimes even appearing **positive**, which contradicts the daily pattern.

This is an important phenomenon to understand, and I explain it carefully in the final analysis.


In [50]:
plot_dual(monthly, "Monthly NYC Subway Ridership vs COVID-19 Cases (Totals)")


## Final Takeaways: Understanding Short-Term Behavior vs. Long-Term Trends

This project reveals a surprisingly rich dynamic between public health conditions and public mobility in New York City. By comparing daily subway ridership with daily COVID-19 case counts, the first and most consistent pattern we observe is a strong **inverse relationship**. When infections rise sharply, subway ridership drops almost immediately; when infections fall, ridership begins to recover. This short-term pattern reflects real, intuitive behavioral reactions: New Yorkers made rapid adjustments to their mobility in response to perceived risk.

However, when we step back and look at the same data through **weekly** and **monthly** lenses, the relationship changes. At these aggregated time scales, the inverse pattern becomes less obvious, and in certain periods—especially before 2023—the two lines even appear to rise together. At first glance, this seems contradictory to the daily behavior. But the explanation lies in the difference between short-term and long-term forces shaping the data.

Daily data captures **behavioral responses**: people changed their subway usage quickly in response to daily news, case spikes, and risk perception. Weekly and monthly data, on the other hand, highlight **structural trends**—the long, slow reopening of the city. Over months and years, NYC gradually returned to normal: offices reopened, schools resumed in-person learning, vaccine uptake increased, and mask mandates eased. This structural recovery caused subway ridership to climb steadily upward, regardless of temporary COVID fluctuations. At the same time, COVID case waves—while still present—became shorter, more irregular, and less behavior-changing over time, especially by late 2022.

When we aggregate data by week or month, these two forces overlap:  
- **Short-term dips in ridership** (caused by case waves) get smoothed out by averaging.  
- **Long-term increases in ridership** (caused by reopening) become the dominant signal.  
Because aggregation mixes pre-wave and post-wave days into the same bins, weekly and monthly charts can unintentionally create the illusion of a positive relationship before 2023—even though the daily reality was clearly inverse.

After mid-2023, the city’s reopening process was largely complete, and pandemic waves had faded. At this point, the long-term trend and the daily behavioral pattern no longer contradicted each other. This is why the long-term (weekly/monthly) charts and short-term (daily) charts begin to show consistent patterns again.

Ultimately, this analysis shows why **time scale matters** in data interpretation. The choice of daily, weekly, or monthly data does not merely change the smoothness of a graph—it changes the apparent relationship between the variables themselves. Daily data reflects immediate human behavior. Weekly and monthly data reflect structural, societal change. Neither is “more correct”; each answers a different question. Understanding the difference allows us to interpret the pandemic and NYC’s mobility recovery more accurately and recognize the complexity behind real-world data.
